In [1]:
import IPython, nussl, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath,math,time,vamp
from IPython.display import Audio, display
from scipy.fftpack import fft, ifft
 
# This line makes sure your plots happen IN the webpage you're building, instead of in separate windows.
%matplotlib inline

In [2]:
n_fft = 2048
hop_length = 512
sr = 11025

# get the signal
signal0, sr = librosa.load(path='datasets/genres/blues/blues.00000.au', sr=sr)
signal1, sr = librosa.load(path='datasets/genres/blues/blues.00001.au', sr=sr)
background0, sr = librosa.load(path='datasets/genres-repet/blues/blues.background.00000.au', sr=sr)
background1, sr = librosa.load(path='datasets/genres-repet/blues/blues.background.00001.au', sr=sr)
foreground0, sr = librosa.load(path='datasets/genres-repet/blues/blues.foreground.00000.au', sr=sr)
foreground1, sr = librosa.load(path='datasets/genres-repet/blues/blues.foreground.00001.au', sr=sr)
harmonic0, sr = librosa.load(path='datasets/genres-hpss/blues/blues.harmonic.00000.au', sr=sr)
harmonic1, sr = librosa.load(path='datasets/genres-hpss/blues/blues.harmonic.00001.au', sr=sr)
percussive0, sr = librosa.load(path='datasets/genres-hpss/blues/blues.percussive.00000.au', sr=sr)
percussive1, sr = librosa.load(path='datasets/genres-hpss/blues/blues.percussive.00001.au', sr=sr)

In [3]:
vamp.list_plugins()

['bbc-vamp-plugins:bbc-energy',
 'bbc-vamp-plugins:bbc-intensity',
 'bbc-vamp-plugins:bbc-peaks',
 'bbc-vamp-plugins:bbc-rhythm',
 'bbc-vamp-plugins:bbc-spectral-contrast',
 'bbc-vamp-plugins:bbc-spectral-flux',
 'bbc-vamp-plugins:bbc-speechmusic-segmenter']

The following functions are used to calculates a 9 dimensional vector corresponding to the musical surface features outlined section 3.1 in Tzanetazis's Automatic Musical Genre Classification Of Audio Signals. The paper can be found by following the link below:
http://www.cs.northwestern.edu/~pardo/courses/eecs352/papers/genre%20classification%20low%20level%20-%20tzanetakis.pdf

In [4]:
def meanAndStandardDeviationCentroid(signal, sample_rate):
    """
    computes the mean and standard deviation centroid, given an audio vector of an audio file
    """
    centroid = vamp.collect(signal,sample_rate,'vamp-libxtract:spectral_centroid')['vector'][1]
    mean = np.mean(centroid)
    standard_dev = np.std(centroid)
    return mean, standard_dev

def meanAndStandardDeviationRolloff(signal, sample_rate):
    """
    computes the mean and standard deviation rolloff, given an audio vector of an audio file
    """
    rolloff = vamp.collect(signal,sample_rate,'vamp-libxtract:rolloff')['vector'][1]
    
    mean = np.mean(rolloff)
    standard_dev = np.std(rolloff)
    return mean, standard_dev
    
def zeroCrossings(signal,sample_rate):
    """
    computes zero crossings value of a signal
    """
    zero_crossings = vamp.collect(signal,sample_rate,'vamp-libxtract:zcr')['vector'][1]
    
    mean = np.mean(zero_crossings)
    standard_dev = np.std(zero_crossings)
    return mean,standard_dev

def flux(signal,sample_rate):
    """
    computes the flux of a signal
    """
    flux = vamp.collect(signal,sample_rate,'bbc-vamp-plugins:bbc-spectral-flux')['vector'][1]
    
    mean = np.mean(flux)
    standard_dev = np.std(flux)
    return mean,standard_dev

def low_energy(signal,sample_rate):
    """
    computes the low_energy value for a signal
    """
    energy = a=vamp.collect(signal,sample_rate,'bbc-vamp-plugins:bbc-energy')['vector'][1]
    
    mean = np.mean(energy)
    count = 0.
    for el in energy:
        if el < mean:
            count += 1
    return count/float(len(energy))

def surface_features(signal, sample_rate):
    """
    Calculates a 9 dimensional vector corresponding to the musical surface features outlined in Tzanetazis's Automatic Musical Genre Classification
    Of Audio Signals
    """
    mean_centroid,std_centroid = meanAndStandardDeviationCentroid(signal, sample_rate)
    mean_rolloff,std_rolloff = meanAndStandardDeviationRolloff(signal, sample_rate)
    mean_zcr,std_zcr = zeroCrossings(signal, sample_rate)
    mean_flux,std_flux = flux(signal, sample_rate)
    low_e = low_energy(signal, sample_rate)
    surface_features = np.array([mean_centroid,std_centroid,mean_rolloff,std_rolloff,mean_zcr,std_zcr,mean_flux,std_flux,low_e])
    return surface_features

The following functions are used to calculate a 8 dimensional vector corresponding to the rhythmic features outlined section 3.2 in Tzanetazis's Automatic Musical Genre Classification Of Audio Signals.

In [9]:
acor = vamp.collect(signal0,sr,'vamp-libxtract:autocorrelation')['matrix'][1]

In [ ]:
surface_features(signal0, sample_rate)